## 2. Uczenie głębokie

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

In [20]:
EPOCHS = 50
BATCH_SIZE = 32
STEPS_PER_EPOCH = 32
VALIDATION_STEPS = 64
MODEL_FILE = 'models/vgg16-last-conv-rmsprop.model'

In [2]:
trdata_gen = ImageDataGenerator()
traindata = trdata_gen.flow_from_directory(directory="../data/Training",target_size=(224,224)) 

Found 67692 images belonging to 131 classes.


In [3]:
tsdata_gen = ImageDataGenerator()
testdata = tsdata_gen.flow_from_directory(directory="../data/Test", target_size=(224,224))

Found 22688 images belonging to 131 classes.


In [4]:
CLASSES = len(traindata.class_indices)

In [5]:
base_model = VGG16(weights='imagenet', include_top = False , input_shape=(224,224, 3))
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
##freeze all  base_model layers and train the last ones
for layer in base_model.layers:
    layer.trainable = False

In [18]:
checkpoint = ModelCheckpoint("vgg16_1_pretrained.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=20, verbose=1, mode='auto')

2.1 Uczenie ostatniej warstwy splotowej i warstwy klasyfikującej

In [11]:
last_conv_flat = Flatten()(base_model.layers[-1].output)
# # add new classifier layers
# class_dense_flat = Dense(1024, activation='relu')(last_conv_flat) -what about this
output = Dense(CLASSES, activation='softmax')(last_conv_flat)
# define new model
## maybe add dropout and maybe one more dense
last_conv_model = Model(inputs=base_model.inputs, outputs=output)
# summarize
last_conv_model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

ValueError: No such layer: functional_7.

In [15]:
last_conv_model.get_layer('block5_conv3').trainable = True
last_conv_model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
last_conv_model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
last_conv_model_history = last_conv_model.fit(
    traindata,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=testdata,
    validation_steps=VALIDATION_STEPS,
    shuffle=True,
    callbacks=[checkpoint, early_stopper]
)


model.save(MODEL_FILE)

Epoch 1/50
32/32 [==============================] - ETA: 0s - loss: 8.9344 - acc: 0.0527
Epoch 00001: val_acc improved from -inf to 0.07910, saving model to vgg16_1_pretrained.h5
32/32 [==============================] - 545s 17s/step - loss: 8.9344 - acc: 0.0527 - val_loss: 4.4004 - val_acc: 0.0791
Epoch 2/50
26/32 [=======================>......] - ETA: 52s - loss: 4.2331 - acc: 0.1310 

2.2 Uczenie dwóch ostatnich warstw splotowych i warstwy klasyfikującej

2.3 Uczenie całej sieci

2.4 Uproszczenie struktury